# MVP SQLi 루프 실험

단일 LLM + 정적 RAG 스냅샷 + 로컬 Docker 실행기를 이용한 최소 생성→검증 루프 실험.

In [1]:
SCENARIO = {
    'scenario_id': 'sid-mvp-sqli-0001',
    'seed': 42,
    'model_version': 'gpt-4.1-mini',
    'prompt_hash': 'sha256:tmpl-mvp-v1',
    'retrieval_snapshot_id': 'rag-snap-mvp-0001',
    'base_image_digest': 'sha256:base-mvp',
    'sbom_ref': 'file://artifacts/sid-mvp-sqli-0001/build/sbom.spdx.json',
    'variation_key': {
        'mode': 'deterministic',
        'top_p': 1.0,
        'temperature': 0.0,
        'self_consistency_k': 1,
        'pattern_pool_seed': 20241106
    }
}
SCENARIO

{'scenario_id': 'sid-mvp-sqli-0001',
 'seed': 42,
 'model_version': 'gpt-4.1-mini',
 'prompt_hash': 'sha256:tmpl-mvp-v1',
 'retrieval_snapshot_id': 'rag-snap-mvp-0001',
 'base_image_digest': 'sha256:base-mvp',
 'sbom_ref': 'file://artifacts/sid-mvp-sqli-0001/build/sbom.spdx.json',
 'variation_key': {'mode': 'deterministic',
  'top_p': 1.0,
  'temperature': 0.0,
  'self_consistency_k': 1,
  'pattern_pool_seed': 20241106}}

In [2]:
import os, random
try:
    import numpy as np
except Exception:
    np = None
try:
    import torch
except Exception:
    torch = None

def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if np is not None:
        np.random.seed(seed)
    if torch is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(SCENARIO.get('seed', 42))
print('Seed fixed:', SCENARIO.get('seed', 42))

Seed fixed: 42


## 1) 요구 정의
- CWE-89(SQLi) 테스트베드 요구
- 기술 스택: Python Flask + SQLite (정적 템플릿)
- 입력 채널: `/login` REST 엔드포인트, 폼 입력 기반
- RAG 근거: Halfond2006 + 내부 SQLi 노트


## 2) 실험 단계 계획
- 자동 실행: `python scripts/mvp_loop.py --input inputs/mvp_sqli.yml`
- 수동 단계:
    1. PLAN → `python orchestrator/plan.py --input inputs/mvp_sqli.yml`
    2. DRAFT → `python agents/generator/main.py --sid sid-mvp-sqli-0001`
    3. BUILD → `scripts/mvp_loop.py` 내부 `build_stage` 참고
    4. RUN → `python workspaces/sid-mvp-sqli-0001/poc/poc.py --log ...`
    5. VERIFY → `python evals/poc_verifier/mvp_sqli.py --log ...`
    6. PACK → `scripts/mvp_loop.py` 내부 `pack_stage`


## 3) 실행 메모
- RAG은 `rag/corpus/processed/mvp-sample` snapshot만 사용
- Variation Key 고정으로 deterministic 모드 유지
- Executor는 로컬 Docker, 네트워크 차단 옵션 `--network none` 적용


## 4) 결과 요약 (기록용)

| 단계 | 상태 | 메모 |
| --- | --- | --- |
| Build | pending | `docker build -f mvp/Dockerfile .` |
| Run | pending | `docker run --rm --network none sid-mvp-sqli-0001` |
| Verify | pending | `python evals/poc_verifier/mvp_sqli.py` |

결과 확정 후 `docs/experiments/`에 요약을 추가한다.
